Naïve Bayes is a simple but powerful classifier based on a probabilistic model
derived from the Bayes' theorem. Basically it determines the probability that an
instance belongs to a class based on each of the feature value probabilities. The naïve
term comes from the fact that it assumes that each feature is independent of the rest,
that is, the value of a feature has no relation to the value of another feature.

Despite being very simple, it has been used in many domains with very good
results. The independence assumption, although a naïve and strong simplification,
is one of the features that make the model useful in practical applications. Training
the model is reduced to the calculation of the involved conditional probabilities,
which can be estimated by counting frequencies of correlations between feature
values and class values.
One of the most successful applications of Naïve Bayes has been within the field
of Natural Language Processing (NLP). NLP is a field that has been much related
to machine learning, since many of its problems can be formulated as a classification
task. Usually, NLP problems have important amounts of tagged data in the form
of text documents. This data can be used as a training dataset for machine
learning algorithms.

dataset consists of around
19,000 newsgroup messages from 20 different topics ranging from politics and
religion to sports and science.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')

In [4]:
print (type(news.data), type(news.target), type(news.target_names))

<class 'list'> <class 'numpy.ndarray'> <class 'list'>


In [5]:
print (news.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [7]:
print (len(news.data))
print (len(news.target))

18846
18846


In [8]:
print (news.data[0])
print (news.target[0], news.target_names[news.target[0]])

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


10 rec.sport.hockey
